----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content layer - Pricing snapshot
This notebook demonstrates how to retrieve Pricing snapshot data.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [2]:
import refinitiv.data as rd
from refinitiv.data.content import pricing

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [3]:
# Open the session named 'platform.rdp' defined in the 'sessions' section of the refinitiv-data.config.json configuration file
rd.open_session('platform.rdp')

<refinitiv.data.session.platform.Definition object at 0x7fc6d7619c50 {session_name='rdp'}>

## Retrieve data

### Get Price Snapshot

In [4]:
response = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_data()

response.data.df

,Instruments,BID,ASK
0,EUR=,1.1331,1.1332
1,GBP=,1.3347,1.3351
2,JPY=,114.14,114.16
3,CAD=,1.2778,1.2782


### Alternative method for Snapshot data
If the above code generates an error, this could be because you are not licensed for the RDP Snapshot API OR you are using a platform session only connected to a Real-Time Distribution System (a.k.a. TREP).
If this is the case, then the following can be used to request a Snapshot from the Streaming data feed (assuming you have a Streaming Data licence) - note the **get_stream()** call

In [5]:
non_streaming = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()

# We want to just snap the current prices, don't need updates
# Open the instrument in non-streaming mode
non_streaming.open(with_updates=False)

# Snapshot the prices at the time of the open() call
non_streaming.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.133,1.1334
1,GBP=,1.335,1.3354
2,JPY=,114.14,114.16
3,CAD=,1.278,1.2784


### Alternative ways of accessing instruments + values

#### Direct Access to fields

In [6]:
non_streaming['EUR=']['BID']

1.133

In [7]:
eur = non_streaming['EUR=']
eur['ASK']

1.1334

#### Iterate on fields

In [8]:
print('JPY=')
for field_name, field_value in non_streaming['JPY=']:
    print(f"\t{field_name} : {field_value}")

JPY=
	BID : 114.14
	ASK : 114.16


#### Iterate on instruments and fields

In [9]:
for instrument in non_streaming:
    print(instrument.name)
    for field_name, field_value in instrument:
        print(f"\t{field_name} : {field_value}")

EUR=
	BID : 1.133
	ASK : 1.1334
GBP=
	BID : 1.335
	ASK : 1.3354
JPY=
	BID : 114.14
	ASK : 114.16
CAD=
	BID : 1.278
	ASK : 1.2784


### Close the stream

In [10]:
non_streaming.close()

<OpenState.Closed: 3>

## Close the session

In [11]:
rd.close_session()